# 드롭아웃 적용 여부 차이 비교 실습

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.datasets as datasets
import torchvision.transforms as transforms

## 드롭아웃 적용한 신경망 모델 정의

In [2]:
class DropoutNet(nn.Module):
    def __init__(self):
        super(DropoutNet, self).__init__()
        self.fc1 = nn.Linear(784,500) # MNIST = 28x28 = 784 -> 1차원
        self.dropout = nn.Dropout(p=0.5)
        self.fc2 = nn.Linear(500,10)
        
    def forward(self, x):
        x = x.view(x.size(0), -1) # 입력 데이터 크기 조절 -> 1차원 으로 펼침
        x = torch.relu(self.fc1(x)) # self.fc1(x) -> relu 적용
        x = self.dropout(x)
        x = self.fc2(x)
        return x

## 드롭아웃 적용하지 않은 신경망 모델 정의

In [3]:
class NonDropoutNet(nn.Module):
    def __init__(self):
        super(NonDropoutNet, self).__init__()
        self.fc1 = nn.Linear(784, 500)
        self.fc2 = nn.Linear(500,10)
        
    def forward(self, x) : 
        x = x.view(x.size(0), -1) # 입력 데이터 크기 조절 -> 1차원 으로 펼침 
        x = torch.relu(self.fc1(x)) # self.fc1(x) -> relu 적용
        x = self.fc2(x)
        
        return x

## MNIST 데이터셋 로드 및 전처리

In [4]:
train_transform = transforms.Compose([
#     transforms.AutoAugment(),
    transforms.ToTensor(),
    ### transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
    transforms.Normalize((0.5,),(0.3,))
])

test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,),(0.3,))
])

train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=train_transform)
test_dataset = datasets.MNIST(root="./data", train=False, download=True, transform=test_transform)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=False)

### 드롭아웃 적용 모델 초기화, loss, 옵티마이저 정의

In [5]:
# 드롭아웃 적용 모델 초기화 
dropout_model = DropoutNet()
print(dropout_model)

# loss 함수, 옵티마이저 설정 
dropout_criterion = nn.CrossEntropyLoss()
dropout_optimizer = optim.SGD(dropout_model.parameters(), lr=0.001)

DropoutNet(
  (fc1): Linear(in_features=784, out_features=500, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc2): Linear(in_features=500, out_features=10, bias=True)
)


### 드롭아웃 적용 모델 학습, 평가 정의

In [6]:
# 드롭아웃 적용 모델 학습 코드 
for epoch in range(10) : 
    dropout_model.train() #### -> train 모드 선언 
    for images, labels in train_loader : 
        dropout_optimizer.zero_grad()
        dropout_output = dropout_model(images)
        dropout_loss = dropout_criterion(dropout_output, labels)
        dropout_loss.backward()
        dropout_optimizer.step()
        
# 드롭아웃 적용 모델 평가 코드 
dropout_model.eval()
with torch.no_grad() : 
    dropout_correct = 0
    dropout_total = 0
    
    for images, labels in test_loader : 
        test_out = dropout_model(images)
        _, dropout_pre = torch.max(test_out.data, 1)
        dropout_total += images.size(0)
        dropout_correct += (dropout_pre == labels).sum().item()
        
    print("드롭아웃 적용 모델 정확도 >> {:.2f}%".format(100 * dropout_correct / dropout_total))

드롭아웃 적용 모델 정확도 >> 91.36%


### 드롭아웃 적용되지 않은 모델 학습 평가 정의

In [7]:
# 모델 호출 
non_dropout_model = NonDropoutNet()

# 옵티마이저 loss function 정의 
non_dropout_criterion = nn.CrossEntropyLoss()
non_dropout_optimizer = optim.SGD(non_dropout_model.parameters(), lr=0.001)

# 드롭아웃 적용되지 않은 모델 학습
# 시작 시간 구하시고 
for epoch in range(10) : 
    non_dropout_model.train()
    for images, labels in train_loader :
        non_dropout_optimizer.zero_grad()
        no_output = non_dropout_model(images)
        no_losss = non_dropout_criterion(no_output, labels)
        no_losss.backward()
        non_dropout_optimizer.step()

# 현재시간 - 시작 시간         
# 드롭아웃 적용하지 않은 모델 평가 
non_dropout_model.eval()
with torch.no_grad()  :
    no_correct = 0
    no_total = 0
    
    for images, labels in test_loader : 
        ouput = non_dropout_model(images)
        _, pred = torch.max(ouput.data, 1)
        no_total += labels.size(0)
        no_correct += (pred == labels).sum().item()
        
    print("드롭아웃 적용하지 않은 모델 정확도 >> {:.2f}%".format(100 * no_correct / no_total))

드롭아웃 적용하지 않은 모델 정확도 >> 91.06%
